# Prática: Redes Neurais Convolucionais

Vamos agora implementar a rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das redes que trouxeram todo esse interesse para a área de *deep learning*.





In [0]:
!pip install mxnet-cu100

# imports basicos
import time, os, sys, numpy as np
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, utils as gutils, data as gdata

# Tenta encontrar GPU
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu()
ctx

## carregando dados

# código para carregar o dataset do MNIST
# http://yann.lecun.com/exdb/mnist/
def load_data_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'mnist')):
    """Download the MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.MNIST(root=root, train=True)
    mnist_test = gdata.vision.MNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# código para carregar o dataset do CIFAR 10
# https://www.cs.toronto.edu/~kriz/cifar.html
def load_data_cifar10(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'cifar10')):
    """Download the MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.CIFAR10(root=root, train=True)
    mnist_test = gdata.vision.CIFAR10(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter
  
# funções básicas
def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n, l = nd.array([0]), 0, 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            # X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            y = y.astype('float32')
            y_hat = net(X)
            l += loss(y_hat, y).sum()
            acc_sum += (y_hat.argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n, l.asscalar() / n
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, ctx,
                   num_epochs):
    print('training on', ctx)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss, ctx)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

## AlexNet

Agora já temos todo o conhecimento necessário para implementar nossa primeira arquitetura moderna.
Vamos implementar a [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das arquiteturas mais famosas dessa nova onda de rede neurais.

<p align="center">
  <img width=700 src="https://www.researchgate.net/profile/Jaime_Gallego2/publication/318168077/figure/fig1/AS:578190894927872@1514862859810/AlexNet-CNN-architecture-layers.png">
</p>

<p align="center">
  <img width=700 src="https://engmrk.com/wp-content/uploads/2018/10/AlexNet_Summary_Table.jpg">
</p>


In [0]:
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001
    
# Implementa sua rede neural aqui
# Voce pode implementar criando uma variavel net e adicionando as camadas net = nn.Sequential()
# Ou pode criar como uma classe
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr, 'wd': wd_lambda})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 2.0776, train acc 0.216, test loss 1.8543, test acc 0.316, time 112.6 sec
epoch 2, train loss 1.8232, train acc 0.327, test loss 1.7190, test acc 0.372, time 109.5 sec
epoch 3, train loss 1.7367, train acc 0.362, test loss 1.6373, test acc 0.403, time 110.0 sec
epoch 4, train loss 1.6363, train acc 0.398, test loss 1.5274, test acc 0.441, time 109.2 sec
epoch 5, train loss 1.5410, train acc 0.437, test loss 1.4531, test acc 0.470, time 110.5 sec
epoch 6, train loss 1.4811, train acc 0.460, test loss 1.3915, test acc 0.492, time 110.3 sec
epoch 7, train loss 1.4316, train acc 0.482, test loss 1.3806, test acc 0.495, time 110.1 sec
epoch 8, train loss 1.3982, train acc 0.492, test loss 1.3537, test acc 0.513, time 110.6 sec
epoch 9, train loss 1.3610, train acc 0.506, test loss 1.3530, test acc 0.513, time 110.4 sec
epoch 10, train loss 1.3409, train acc 0.515, test loss 1.3089, test acc 0.527, time 110.5 sec
epoch 11, train loss 1.3094, train acc 0